In [1]:
import keras
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_KZsnXxufkWyw9Rqjy3Nnsx9iz9Fx1j_EWwj4eJW94qr',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfire-donotdelete-pr-hskyzwal3dw1ot'
object_key = 'Project archive.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [5]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   rotation_range=180,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
x_train = train_datagen.flow_from_directory(r'./Dataset/Dataset/train_set/',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='binary')

Found 436 images belonging to 2 classes.


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten

In [16]:
model = Sequential()
model.add(Convolution2D(32, (3,3), input_shape=(128, 128, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(150,activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [17]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [18]:
model.fit(x_train, steps_per_epoch=14, epochs=10, validation_data=x_test, validation_steps=4)

Epoch 1/10
14/14 [==============================] - 30s 2s/step - loss: 3.7117 - accuracy: 0.5894 - val_loss: 0.3764 - val_accuracy: 0.9174
Epoch 2/10
14/14 [==============================] - 26s 2s/step - loss: 0.9340 - accuracy: 0.7913 - val_loss: 0.2195 - val_accuracy: 0.9421
Epoch 3/10
14/14 [==============================] - 27s 2s/step - loss: 0.4349 - accuracy: 0.8578 - val_loss: 0.1417 - val_accuracy: 0.9587
Epoch 4/10
14/14 [==============================] - 27s 2s/step - loss: 0.2785 - accuracy: 0.8853 - val_loss: 0.1171 - val_accuracy: 0.9587
Epoch 5/10
14/14 [==============================] - 26s 2s/step - loss: 0.2127 - accuracy: 0.9014 - val_loss: 0.1096 - val_accuracy: 0.9587
Epoch 6/10
14/14 [==============================] - 26s 2s/step - loss: 0.1856 - accuracy: 0.9128 - val_loss: 0.1012 - val_accuracy: 0.9669
Epoch 7/10
14/14 [==============================] - 26s 2s/step - loss: 0.1935 - accuracy: 0.9266 - val_loss: 0.0921 - val_accuracy: 0.9752
Epoch 8/10
14/14 [==

In [21]:
model.save("Forest.h5")

In [22]:
!tar -zcvf model.tgz Forest.h5

Forest.h5


In [23]:
!pip install watson-machine-learning-client

     |████████████████████████████████| 538 kB 27.4 MB/s eta 0:00:01


In [24]:
from ibm_watson_machine_learning import APIClient

API_KEY = "n8h8b7BvZ92GT1tfJQGcgPAwHn-PejzK4EGRMUQ9i6QK"

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(credentials)

In [25]:
client

In [36]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [37]:
space = client.spaces.get_details()
space

{'resources': []}

In [46]:
space_uid = guid_from_space_name(client, 'Forest fire detection')
print("Space UID: ", space_uid)

Space UID:  da336b7d-54d2-488b-bb19-6a54b9df2315


In [47]:
client.set.default_space(space_uid)

'SUCCESS'

In [48]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [49]:
model_details = client.repository.store_model(model="model.tgz", meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid
})

model_id = client.repository.get_model_id(model_details)

In [50]:
client.repository.download(model_id, "model.tar.gz")

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'